# Master video notebook!
This title might be a bit ambitious, but this notebook is supposed to be able to do all of the administration work when it comes to analysing videos. Of course, many functions are already inherent to the kymo_class file and the plot_data file. Ultimately, this notebook is about creating a file hierarchy for the analysis files.

In step one, the Dropbox is scoured for information about videos. If the videos do not have a VideoInfo.txt, the program will look for a .csv, if there is no .csv, the program will look for a .xlsx file. This is currently in conflict with what is happening in the kymo_class.py file, so that one will have to be amended.

### Let's say that there are three ways to initiate a kymograph class:
1. No info file is submitted, and the class will look for such a file itself.
2. An info file is submitted, and the class will use the data in there.
3. A kymograph is submitted, and the class will instantiate with the parameters that are passed with the kymograph.

TODO: Make it so in the kymo_class
TODO: Streamline the variable storage such that edge properties are stored in the edge_analysis class, and video properties are stored in the video_analysis class.

### Below code:
Are just import statements

In [2]:
from IPython.display import clear_output
import re
from amftrack.pipeline.development.high_mag_videos.high_mag_analysis import (
    HighmagDataset,
    VideoDataset,
    EdgeDataset
)
from amftrack.pipeline.development.high_mag_videos.kymo_class import *
from amftrack.pipeline.development.high_mag_videos.plot_data import (
    save_raw_data,
    plot_summary,
    read_video_data
)
import sys
import os
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
from tifffile import imwrite
from tqdm import tqdm
%matplotlib widget
%load_ext autoreload
%autoreload 2
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    clean_degree_4,
)
import scipy
import matplotlib as mpl

from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer

import dropbox
from amftrack.util.dbx import upload_folders, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, download, get_dropbox_folders, get_dropbox_video_folders
from subprocess import call
import logging
import datetime
import glob
import json
from amftrack.pipeline.launching.run_super import run_parallel_flows


logging.basicConfig(stream=sys.stdout, level=logging.debug)
mpl.rcParams['figure.dpi'] = 300


## File declaration
As this notebook is designed to work with Snellius, two items to separate are the raw video files and the anaylsis. The raw video files are large, bulky and not so easy to flip through. Ideally, the video files would be downloaded and the analysis would be stored on a separate folder structure entirely. That way, large scale analysis of analysis folders can happen when there are thousands of videos in the dataset, without having to have those raw video folders on hand.

### Input and output:
Please give separately the folder where raw video data is stored, and where the analysis will be stored. 
Also give the dropbox address of the dataset you want to analyze, this is the first filtering step.

In [3]:
videos_folder = "F:\\AMOLF_Data\\videos\\"
analysis_folder = "F:\\AMOLF_Data\\analysis\\"

In [4]:
# dropbox_address = "/DATA/FLUORESCENCE/DATA_NileRed/"
# dropbox_address=  "/DATA/MYRISTATE/DATA/"
# dropbox_address = "/DATA/TransportROOT/DATA/"
dropbox_address = "/DATA/CocoTransport/"

## Where to go?
If you want to download videos:
Use MODULE 1

If you want to analyze already downloaded videos:
Skip MODULE 1, use MODULE 2

# Module 1: Downloading videos from DropBox

## Dropbox scrounging
The below code is meant to scour the dropbox for information files on the videos. It is also to create a list of the videos within a certain database. The expectation at the very least is that one plate contains many videos, all labeled with a number. The code will take this list, and recreate the hierarchy within the Analysis_Output folder.

### Input:
The input will be the highest folder of the dropbox that needs to be analyzed. You can also set REDO_SCROUNGING to True if you want the dropbox scrounging to happen no maatter what.

### Output
The output will be a DataFrame that can be filtered in the next code block to prepare for downloading. In addition, .json files will also be created for the specific file that is recorded in the input. That way, scrounging the dropbox only has to be done once. 

In [5]:
REDO_SCROUNGING = False

analysis_json = f"{analysis_folder}{dropbox_address[6:]}all_folders_drop.json"
if os.path.exists(analysis_json):
    all_folders_drop = pd.read_json(analysis_json)
excel_json = f"{analysis_folder}{dropbox_address[6:]}excel_drop.json"
if os.path.exists(excel_json):
    excel_drop = pd.read_json(excel_json, typ='series')
if not os.path.exists(analysis_json) or REDO_SCROUNGING:
    all_folders_drop, excel_drop, txt_drop = get_dropbox_video_folders(dropbox_address, True)

    clear_output(wait=False)
    
    excel_addresses = np.array([re.search("^.*Plate.*\/.*Plate.*$", entry, re.IGNORECASE) for entry in excel_drop])
    excel_addresses = excel_addresses[excel_addresses != None]
    excel_addresses = [address.group(0) for address in excel_addresses]
    excel_drop = np.concatenate([excel_addresses,txt_drop])
    if not os.path.exists(f"{analysis_folder}{dropbox_address[6:]}"):
        os.makedirs(f"{analysis_folder}{dropbox_address[6:]}")
    all_folders_drop.to_json(analysis_json)
    pd.Series(excel_drop).to_json(excel_json)
    print(all_folders_drop['tot_path_drop'][0])
    
all_folders_drop 

,folder,Plate number,Date Imaged,tot_path_drop,video
0,Img,403,20230703,DATA/CocoTransport/20230703_Plate403/001/Img,Img


In [6]:
info_addresses  = []
for address in excel_drop:
    csv_name_len = len(address.split(os.sep)[-1])
    if not os.path.exists(analysis_folder + address[6:-csv_name_len]):
        os.makedirs(analysis_folder + address[6:-csv_name_len])
    if not os.path.exists(analysis_folder + address[6:]):
        download(address, analysis_folder + address[6:])
    info_addresses.append(analysis_folder + address[6:])
clear_output(wait=False)
print("All files downloaded!")

All files downloaded!


In [7]:
merge_frame = read_video_data(info_addresses, all_folders_drop)
merge_frame

100%|██████████| 1/1 [00:00<00:00, 81.79it/s]


,imaging_day,storage_path,plate_id,root,strain,treatment,crossing_day,video_int,time_(s),mode,...,Fiber Led,xpos,ypos,zpos,unique_id,tot_path,tot_path_drop,record_time,days_after_crossing,magnification
0,20230703,Dropbox\DATA\CocoTransport,20230703_Plate403,Carrot,C2,001P100N100C,20230626,1,30.0,BF,...,On,12.266,0.32,5.993,AMOLF_Data_analysis,rt/20230703_Plate403/001/Img/,DATA/rt/20230703_Plate403/001/Img/,16:45:29,7,50.0


## Dropbox filtering
If you want all videos in a plate or dataset, you can run this block and forget about it.
Otherwise, this block is where filtering can take place to only download videos with certain properties, like imaging mode. Then a file structure will be created in the videos and analysis folder, and videoInfo.txt folders are created for every video, in the analysis folder. The intent here is to have a uniform Analysis folder structure that works with the Morrison setup.

### Input
Use the section between commented lines to filter the DataFrame, otherwise leave blank
### Output
Within the video and analysis folder, a hierarchy will be created to mimic that of the dropbox folder structure, using the filtered DataFrame. Inside the analysis folder hierarchy, the VideoInfo.txt file will be generated. Either from an existing videoInfo.txt, or the excel/csv sheets.

In [11]:
#####################################################################################
### This is where you can apply the filters. Only those videos will be downloaded ###
#####################################################################################

download_frame = merge_frame

#####################################################################################
### Below code will prepare for those videos to be downloaded into videos_folder  ###
#####################################################################################

download_frame = download_frame.rename(columns={'tot_path' : 'folder'})
download_frame = download_frame.sort_values('unique_id')
download_frame = download_frame.reset_index(drop=True)
download_frame = download_frame.loc[:,~download_frame.columns.duplicated()].copy()

download_frame

,imaging_day,storage_path,plate_id,root,strain,treatment,crossing_day,video_int,time_(s),mode,...,Fiber Led,xpos,ypos,zpos,unique_id,folder,tot_path_drop,record_time,days_after_crossing,magnification
0,20230703,Dropbox\DATA\CocoTransport,20230703_Plate403,Carrot,C2,001P100N100C,20230626,1,30.0,BF,...,On,12.266,0.32,5.993,AMOLF_Data_analysis,rt/20230703_Plate403/001/Img/,DATA/rt/20230703_Plate403/001/Img/,16:45:29,7,50.0


In [13]:
download_frame['analysis_folder'] = [np.nan for i in range(len(download_frame))]
download_frame['videos_folder'] = [np.nan for i in range(len(download_frame))]
# download_frame = download_frame.drop(columns=['index'], axis=1)

for index, row in download_frame.iterrows():
    target_anals_file = f"{analysis_folder}{row['folder'][:-4]}"
#     print(target_anals_file)
    target_video_file = f"{videos_folder}{row['folder']}"
    
    row.loc['analysis_folder'] = target_anals_file
    row.loc['videos_folder'] = target_video_file
    
    if not os.path.exists(target_anals_file):
        os.makedirs(target_anals_file)
    if not os.path.exists(target_video_file):
        os.makedirs(target_video_file)
#     print(row.index)
    row.to_json(f"{target_anals_file}/video_data.json", orient="index")

## Downloading
This section, there is one block of code that will ask you one last time whether all of the parameters are correct. The block of code after that will initiate Snellius jobs to download the videos in the DataFrame from the dropbox. Downloading videos is not that costly, but of course we prefer it to be done as efficiently as possible.
### Input:
Nothing
### Output:
Print statement with the DataFrame and the folders where everything will be stored.
Subsequent block of code will download raw video files to the videos folder.

In [15]:
print("Below videos will be downloaded!")
download_frame

Below videos will be downloaded!


,imaging_day,storage_path,plate_id,root,strain,treatment,crossing_day,video_int,time_(s),mode,...,ypos,zpos,unique_id,folder,tot_path_drop,record_time,days_after_crossing,magnification,analysis_folder,videos_folder
0,20230703,Dropbox\DATA\CocoTransport,20230703_Plate403,Carrot,C2,001P100N100C,20230626,1,30.0,BF,...,0.32,5.993,AMOLF_Data_analysis,rt/20230703_Plate403/001/Img/,DATA/rt/20230703_Plate403/001/Img/,16:45:29,7,50.0,NaN,NaN


In [17]:
run_parallel_transfer(
    "from_drop.py",
    [videos_folder],
    download_frame,
    1,
    "10:00:00",
    "transfer_test"
)
clear_output(wait=False)

print("Sent all the jobs! Use the command '$ squeue' in the terminal to see the progress")

Sent all the jobs! Use the command '$ squeue' in the terminal to see the progress


# Module 2: Analysis
Now that the files have been downloaded, it's time to analyse them. In the below code, you'll be able to either do a complete survey of the analysis folder for as many videos as possible, or use the DataFrame of recently downloaded videos to filter for the videos you want to analyse.

### Input:
DataFrame filters of all videos to be analysed
### Output:
Print statements for all parameters of the analysis session that is about to take place.

In [33]:
print(dropbox_address)

/DATA/MYRISTATE/DATA/


In [8]:
folder_filter = dropbox_address[5:]

img_infos = glob.glob(f"{analysis_folder}{folder_filter}/**/video_data.json", recursive=True)
vid_anls_frame = pd.DataFrame()
for address in img_infos:
    add_info = pd.read_json(address, orient='index').T
    vid_anls_frame = pd.concat([vid_anls_frame, add_info], ignore_index=True)

vid_anls_frame = vid_anls_frame.sort_values('unique_id').reset_index(drop=True)
# print(vid_anls_frame.columns)
# print([(type(entry), entry) for entry in vid_anls_frame.iloc[0]])
vid_anls_frame

,imaging_day,storage_path,plate_id,root,strain,treatment,crossing_day,video_int,time_(s),mode,...,ypos,zpos,unique_id,folder,tot_path_drop,record_time,days_after_crossing,magnification,analysis_folder,videos_folder
0,20230703,Dropbox\DATA\CocoTransport,20230703_Plate403,Carrot,C2,001P100N100C,20230626,1,30.0,BF,...,0.32,5.993,20230703_Plate403_001,CocoTransport/20230703_Plate403/001/Img/,DATA/CocoTransport/20230703_Plate403/001/Img/,16:45:29,7,50.0,/gpfs/home6/svstaalduine/Analysis/CocoTranspor...,/gpfs/scratch1/shared/amftrackflow/videos/Coco...


In [9]:
####################################################################################
### This is where you can apply the filters. Only those videos will be analyzed. ###
####################################################################################

analysis_frame = vid_anls_frame

####################################################################################
### Below code will prepare for those videos to be downloaded to videos_folder.  ###
####################################################################################

print(analysis_frame['unique_id'].to_string())

0    20230703_Plate403_001


## Run Analysis job
Two options: For small analysis, use the first block. This will just do the calculations on the machine. For large-scale analysis, use the second block, as it will create a Snellius job.
## Input:
Snellius job parameters
## Output:
Analysis folder will be populated with analysis tiffs and csv sheets.

In [ ]:
### LARGE VIDEO ANALYSIS

nr_parallel = np.min([len(analysis_frame.index), 16])

run_parallel_flows(
    "flux_extract.py",
    [analysis_folder, 15, 0.95, 0.005, 60, dropbox_address],
    analysis_frame,
    nr_parallel,
    "2:00:00",
    "flux_extract"
)

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1.0 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3053716


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1.0 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3053717


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1.0 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


...

Submitted batch job 3053763


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1.0 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3053764


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 128 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 4 jobs.
sbatch: By default shared jobs get 1792 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 1.0 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 3053765


# Module 3: Bulk Analysis
## First part: Assemble Edge DataFrame


In this initial part of the bulk analysis, all of the analysis folders will be looked through to find the edge data we're looking for. Additionally, there is an optional part to download the analysis folder back to the analysis folder we specified right at the top.

In [14]:
hannah_set = HighmagDataset(download_frame)
print(f"{analysis_folder}{download_frame['folder'][0][:-4]}")
download_frame['videos_folder'][0]

F:\AMOLF_Data\analysis\rt/20230703_Plate403/001/


nan

## Assuming all the videos are already downloaded:
You can use below code to read the video_data.json files that are created during indexing of all the videos